In [29]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_holistic = mp.solutions.holistic  # MediaPipe holistic model
mp_drawing = mp.solutions.drawing_utils

# Function to extract keypoints
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,lh,rh])

def mediapipe_detection(image, model):
    # so opencv reads the image in form of bgr but for detection using mediapipe we require the format to be RGB
    # so cv2.cvtcolor helps in recolouring the image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable this helps us in saving a bit of memory
    # so here image is going to be a frame from video
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    # so that opencv can produce results in bgr format
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
mp_holistic = mp.solutions.holistic # Holistic model
# used for drawing the utilities - points and structure from the midiapipe which is its main function through .drawing_utils
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
imagepath="2.jpg"
frame = cv2.imread(imagepath)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    image, results = mediapipe_detection(frame, holistic)
                    keypoints = extract_keypoints(results)
    
                    # Save keypoints
                    np.save("1.npy", keypoints)

In [30]:
SEQUENCE_LENGTH=1
X=[]
sequence = [np.load("1.npy")]
X.append(sequence)
X=np.array(X)
X.shape

(1, 1, 258)

In [31]:
import tensorflow as tf
model = tf.keras.models.load_model('model.keras')
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                       │ (None, 1, 64)               │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 7)                   │             231 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 712,151 (2.72 MB)

 Trainable params: 237,383 (927.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 474,768 (1.81 MB)

In [32]:
Y_pred = model.predict(X)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
label=['bring water for me',
 'can i help you',
 'hi how are you',
 'i am hungry',
 'i am sitting in the class',
 'i am suffering from fever',
 'i am tired']
label[Y_pred_classes[0]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 876ms/step


'hi how are you'